In [1]:
# Codeblock 1
# pip install mitiq

In [2]:
# Codeblock 2 and 3
import mitiq
mitiq.about()


Mitiq: A Python toolkit for implementing error mitigation on quantum computers
Authored by: Mitiq team, 2020 & later (https://github.com/unitaryfund/mitiq)

Mitiq Version:	0.10.0dev

Core Dependencies
-----------------
Cirq Version:	0.10.0
NumPy Version:	1.20.3
SciPy Version:	1.4.1

Optional Dependencies
---------------------
PyQuil Version:	2.28.1
Qiskit Version:	None
Braket Version:	1.5.16

Python Version:	3.8.0
Platform Info:	Darwin (x86_64)


In [3]:
!pip install pyquil==2.21

     |████████████████████████████████| 208 kB 296 kB/s eta 0:00:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Using cached antlr4_python3_runtime-4.7.2-py3-none-any.whl
  Using cached decorator-4.4.2-py2.py3-none-any.whl (9.2 kB)
  Created wheel for pyquil: filename=pyquil-2.21.0-py3-none-any.whl size=246052 sha256=c3f1b7d3440068b56cca77c280d246e251ea7af3b76967c1a4b25219a1a8cee4
  Stored in directory: /Users/nathanshammah/Library/Caches/pip/wheels/97/86/95/d903bbc63471fb73ac4f2f046eb5291fd4ea59cf112a62ce93
Successfully built pyquil
  Attempting uninstall: decorator
    Found existing installation: decorator 5.0.9
    Uninstalling decorator-5.0.9:
      Successfully uninstalled decorator-5.0.9
  Attempting uninstall: pyquil
    Found existing installation: pyquil 2.28.1
    Uninstalling pyquil-2.28.1:
      Successfully uninstalled pyquil-2.28.1


In [8]:
# Codeblock 4
def executor(circuit: mitiq.QPROGRAM) -> float:
    #some code
    return 

In [9]:
# Codeblock 4 modified

import numpy as np
from cirq import Circuit, depolarize
from cirq import LineQubit, X, DensityMatrixSimulator

SIMULATOR = DensityMatrixSimulator()
# 0.1% depolarizing noise
NOISE = 0.001

def executor(circ: Circuit) -> float:
    """ Simulates a circuit with depolarizing noise at level NOISE.
    Args:
        circ: The quantum program as a cirq object.

    Returns:
        The expectation value of the |0> state.
    """
    circuit = circ.with_noise(depolarize(p=NOISE))
    rho = SIMULATOR.simulate(circuit).final_density_matrix
    # define the computational basis observable
    obs = np.diag([1, 0])
    expectation = np.real(np.trace(rho @ obs))
    return expectation

In [10]:
# Codeblock 4.bis modified 
from cirq import Circuit, LineQubit, X

qbit = LineQubit(0)
circuit = Circuit(X(qbit) for _ in range(80))

In [12]:
# Codeblock 5
from mitiq.zne import execute_with_zne

zne_value = execute_with_zne(circuit, executor)

In [13]:
# Codeblock 6
zne_value = execute_with_zne(circuit, executor)#,scale_noise=<noise scaling method >,factory=<inference method >,)

In [14]:
# Codeblock 7
import  cirq

qreg = cirq.LineQubit.range(2)
circ = cirq.Circuit(
    cirq.ops.H.on(qreg[0]),cirq.ops.CNOT.on(qreg[0], qreg[1])
)
print("Original  circuit:", circ , sep="\n")

Original  circuit:
0: ───H───@───
          │
1: ───────X───


In [15]:
# Codeblock 8
from mitiq.zne import scaling
folded = scaling.fold_gates_from_left(
circ , scale_factor =2. )
print("Folded circuit:", folded , sep="\n")

Folded circuit:
0: ───H───H───H───@───
                  │
1: ───────────────X───


In [16]:
# Codeblock 9
folded = scaling.fold_gates_from_right( circ , scale_factor =2.
)
print("Folded circuit:", folded , sep="\n")
# Folded circuit:
# 0: ---H---@---@---@---
 # |||
# 1: -------X---X---X---

Folded circuit:
0: ───H───@───@───@───
          │   │   │
1: ───────X───X───X───


In [17]:
# Codeblock 10
from mitiq.zne.scaling import fold_global
folded = fold_global(circ, scale_factor=3.) 
print("Folded circuit:", folded , sep="\n")
# Folded circuit:
# 0: ---H---@---@---H---H---@---
#|||
# 1: -------X---X-----------X---

Folded circuit:
0: ───H───@───@───H───H───@───
          │   │           │
1: ───────X───X───────────X───


In [18]:
# Codeblock 11
from mitiq.zne import execute_with_zne 
from mitiq.zne.scaling import fold_global

zne_value = execute_with_zne(circuit, 
                             executor,
                             scale_noise=fold_global )

In [19]:
# Codeblock 12
from mitiq.zne.inference import LinearFactory
linear_factory = LinearFactory( scale_factors=[1.0, 2.0, 3.0],
)

In [20]:
# Codeblock 13
from mitiq.zne import execute_with_zne
zne_value = execute_with_zne(
circuit , executor ,
factory=linear_factory )

In [21]:
# Codeblock 14
from mitiq.zne import execute_with_zne
from mitiq.zne.inference import PolyFactory
zne_value = execute_with_zne(
circuit , executor ,
factory=PolyFactory(
scale_factors=[1.0, 2.0, 3.0], order=2
) )

In [22]:
# Codeblock 15
from mitiq.zne import execute_with_zne
from mitiq.zne.inference import AdaExpFactory
zne_value = execute_with_zne( circuit ,
executor , factory=AdaExpFactory(
scale_factor=2.0, steps=5 )
)

In [23]:
# Codeblock 16

import numpy as np
from mitiq.zne.inference import BatchedFactory

class MyFactory(BatchedFactory): 
    def reduce(self) -> float:
        # Get scale factors and exp values
        scale_factors = self.get_scale_factors()
        exp_vals = self.get_expectation_values()
        # Define the custom fit here!
        coeffs = np.polyfit(
        scale_factors , exp_vals , deg=2 )
        zne_value = coeffs[-1]
        # Return the ZNE value
        return np.clip(zne_value , -1.0, 1.0)

In [24]:
# Codeblock 17

import qiskit
provider = qiskit.IBMQ.load_account()
def executor(
    circuit: qiskit.QuantumCircuit , backend: str = "ibmq_london",
    shots: int = 1024 ) -> float:
    # Execute the circuit
    job = qiskit.execute(
    experiments=folded_circuits , backend=provider.get_backend(backend),
    optimization_level=0, shots=shots
    )
    # Get the measurement data
    counts = job.result().get_counts()
    # Return the observable
    return counts["00"] / shots

In [25]:
import pyquil
from mitiq.mitiq_pyquil.compiler import basic_compile
aspen8 = pyquil.get_qc("Aspen -8", as_qvm=False)
def executor(program: pyquil.Program , shots: int = 1024) -> float:
    # Make a copy of the program
    prog = program.copy()
    # Get list of qubits used in the program
    qubits = prog.get_qubits()
    # Add classical memory declaration
    ro = prog.declare("ro", "BIT", len(qubits))
    # Add measurement operations
    for idx, q in enumerate(qubits):
        prog += MEASURE(q, ro[idx])
    # Add number of shots
    prog.wrap_in_numshots_loop(shots)
    # Compile the program, keeping folded gates
    prog = basic_compile(prog)
    # Convert to an executable and run
    executable = aspen8.compiler. native_quil_to_executable(prog)
    results = aspen8.run(executable)
    # Return the observable
    all_zeros = [sum(b) == 0 for b in results]
    return sum(all_zeros) / shots

ModuleNotFoundError: No module named 'mitiq.mitiq_pyquil.compiler'

In [ ]:
---------------------------------------------------------------------------
ModuleNotFoundError                       Traceback (most recent call last)
<ipython-input-31-5c7bca2d8478> in <module>
      1 import pyquil
----> 2 from mitiq.mitiq_pyquil.compiler import basic_compile
      3 aspen8 = pyquil.get_qc("Aspen -8", as_qvm=False)
      4 def executor(program: pyquil.Program , shots: int = 1024) -> float:
      5     # Make a copy of the program

~/github/mitiq/mitiq/mitiq_pyquil/__init__.py in <module>
     14 # along with this program.  If not, see <https://www.gnu.org/licenses/>.
     15 
---> 16 from mitiq.mitiq_pyquil.conversions import (
     17     from_pyquil,
     18     from_quil,

~/github/mitiq/mitiq/mitiq_pyquil/conversions.py in <module>
     20 from pyquil import Program
     21 
---> 22 from cirq.contrib.quil_import import circuit_from_quil
     23 
     24 QuilType = str

ModuleNotFoundError: No module named 'cirq.contrib.quil_import'
 with pyquil 2.18

In [ ]:
! pip install pyquil

In [26]:
# Codeblock 19
import  cirq
dsim = cirq.DensityMatrixSimulator ()

def  executor(
    circ: Circuit ,
    obs: cirq.PauliString ,
    noise: float = 0.01,
    shots: int = 102410) -> float:
    # Add  depolarizing  noise  to the  circuit
    noisy = circ.with_noise(
        cirq.depolarize(p=noise)
    )
    
    # Do the  sampling
    
    psum = cirq.PauliSumCollector(
        noisy ,
        obs ,
        samples_per_term=shots
    )
    psum.collect(sampler=dsim)
    
    # Return  the  expectation  value
    return  psum.estimated_energy ()


In [27]:
# Codeblock 20
from functools import partial
def executor(qprogram, arg1, arg2) -> float: 
    ...
    new_executor = partial(
    executor , arg1=arg1value ,
    arg2=arg2value )

In [28]:
# Codeblock 21

from mitiq.zne import mitigate_executor

mitigated_executor = mitigate_executor(
    executor ,
    scale_noise = <noise scaling method>,
    factory = <inference method> )

zne_value = mitgated_executor(circuit)

SyntaxError: invalid syntax (<ipython-input-28-29b70bdddf8a>, line 7)

In [29]:
# Codeblock 22

from mitiq import QPROGRAM
from mitiq.zne import zne_decorator

@zne_decorator(
    #factory=<inference method >, scale_noise=<noise scaling method >
    )

def executor(circuit: QPROGRAM) -> float:
    ...

zne_value = executor(circuit)

TypeError: unsupported operand type(s) for /: 'NoneType' and 'int'